# Benzinga-Nachrichten-Verarbeitung

## Setup up

### Cluster spin up

In [1]:
use_colab = False
if use_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    cwd="/content/drive/MyDrive/NewsTrading/trading_bot"
    %cd /content/drive/MyDrive/NewsTrading/trading_bot
    %pip install -r requirements_clean.txt

In [2]:
%cd /gxfs_work/cau/sunms534/trading_bot/

/gxfs_work/cau/sunms534/trading_bot


/gxfs_work/cau/sunms534/.conda/envs/my_pytorch_env/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [61]:
import os
import re
import pandas as pd
from src.config import config 
from tqdm import tqdm
tqdm.pandas()

import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

from src.utils.dataframes import parallelize_dataframe
from src.preprocessing.news_parser import infer_author, filter_body, body_formatter

pd.set_option(
    'display.max_colwidth', 2000
)

## Grobes HTML-Parsing
Als erstes müssen wir die HTML-Dokumente zu normalem Text umwandeln, ansonsten sind die Text-Zellen zu groß und führen zu Problemen mit PyArrow/Dask.

In [ ]:
def body_formatter_for_series(s: pd.Series):
    return s.progress_apply(body_formatter)

In [ ]:
df = pd.read_parquet(config.data.benzinga.raw)
df["html_body"] = parallelize_dataframe(df["html_body"], body_formatter_for_series, n_cores=os.cpu_count())
df = df.rename(columns={"html_body":"body"})

In [ ]:
df.to_parquet(config.data.benzinga.raw_html_parsed)

## Author-Inferenz

In [ ]:
ddf = pd.read_parquet(config.data.benzinga.raw_html_parsed)

In [ ]:
# Remove rows for which noo stock ticker is recorded
ddf = ddf[ddf.stocks != '']

In [ ]:
# Convert `channels`  datatype from string to list
ddf["channels"] = ddf["channels"].progress_apply(eval)

Untersuche als nächstes die Behauptung, dass **PRNewswire** und **Businesswire** den gesamten Markt für Pressemeldungen in den USA kontrollieren. Wenn dem so ist, und sie nicht noch weitere, unwichtige Meldungen veröffentlichen, dann können wir einfach die Newsartikel nach diesen Autoren filtern und uns viel Arbeit ersparen.

In [ ]:
%%time
ddf["inferred_author"] = None
ddf["inferred_author"] = ddf.body.progress_apply(infer_author, meta=pd.Series(dtype="string"))

In [ ]:
# value_counts for authors
auhtor_value_counts = pd.concat([ddf.author.value_counts().head(10), ddf.inferred_author.value_counts().head(10)], axis=1)

In [ ]:
auhtor_value_counts

In [ ]:
auhtor_value_counts.sum().diff()

Ungefähr 650k Nachrichten werden ausgelassen, wenn nur die vier Hauptvertreiber von Pressemeldungen berücksichtigt werden.

In [ ]:
ddf = ddf[~ddf.inferred_author.isna()]

In [ ]:
ddf["inferred_author"] = ddf["inferred_author"].astype("string")

In [ ]:
ddf["channels"] = ddf.channels.progress_apply(lambda x: str(x))

In [ ]:
ddf.inferred_author.value_counts().compute()

In [ ]:
ddf.inferred_author.value_counts().sum().compute()

In [ ]:
ddf = ddf.drop(columns=["author"]).rename(columns={"inferred_author":"author"})

In [ ]:
# Contains 100k rows
earnings_ddf = ddf[ddf.channels.apply(lambda x: "Earnings" in x, meta=pd.Series(dtype=bool))]

In [ ]:
# value counts for authors of earnings reports (contrast to value counts of all news articles)
earnings_ddf.inferred_author.value_counts().head(10)

Hier sehen wir, dass es keine einzige Pressemeldung von **Business Wire** gibt, die mit *Earnings* gekennzeichnet sind. Trotzdem gibt es relevante *Earnings* reports von Business Wire. Dies habe ich kurz verifiziert...

Wie viele Nachrichten bleiben, wenn wir auf relevante Ticker filtern? Wir wollen nicht(!) - so ist es momentan - auf die momentane Russell 3k-Zusammensetzung filtern, denn wir wollen auch ungelistete bzw. ehemalige Russell-Aktien beachten.


**1. Full-Name-Discovery:**

Herausfinden des vollen Namens des Unternehmens für jeden Ticker, damit 1. der Text richtig geparst werden kann und 2. damit wir einen Anhaltspunkt für das Ticker-Grouping haben.


**2. Ticker-Filtering:**

Alle Ticker herausfiltern, die wir nicht brauchen. Wenn wir aber ein großes Aktienuniversum (mit inzwischen ungelisteten Aktien) benutzen, werden wir fast alle Nachrichten behalten können. Allerdings lassen sich so Fehlerhafte Nachrichten/Ticker etc. herausfiltern.


**3. Ticker-Grouping:**

Was machen wir, wenn wir mehrerer Aktiengattungen für ein Unternehmen haben? Z.B. Vorzugs- und Stammaktien. Wir können i.A. die Stammaktie nehmen, da diese normalerweise ein höheres Handelsvolumen aufweist. D.h. wir bilden alle Ticker der Benzinga-Nachrichten auf den Ticker der Stammaktie ab.


**4. Firmennamen-Nachrichtenkörper-Verifikation:**

Da Ticker wiederverwendet werden können bzw. sich verändern können wollen wir sicherstellen, dass der Unternehmensname im Nachrichtenkörper vorkommt. Bzw. generell ist das eine gute Datensäuberungs-Maßnahme. Einerseits verhindern wir,
dass später Aktienkurse einer falschen Aktie zugeordnet wird. Andererseits werden dadurch evtl. auch weniger seriöse Nachrichten herausgefiltert, die nicht
die Kontaktadresszeile des Unternehmens am Ende besitzen, in dem der vollständige Unternehmensname vorkommt.

In [6]:
ddf = ddf[ddf.stocks.isin(mapper.index.to_list())]
ddf["company_name"] = ddf.stocks.progress_map(lambda x: mapper.company_names.loc[x]).astype(str)
ddf["short_name"] = ddf.stocks.progress_map(lambda x: mapper.short_name.loc[x]).astype(str)
print(f"Es verbleiben {ddf.shape[0]} Nachrichten, für die wir den Ticker zu einem Firmennamen aufgelösen konnten.")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1017673/1017673 [00:08<00:00, 120427.61it/s]


Es verbleiben 1017673 Nachrichten, für die wir den Ticker zu einem Firmennamen aufgelösen konnten.


In [7]:
ddf.company_name

__null_dask_index__
74407                             Rent the Runway, Inc.
74409                            WW International, Inc.
74416                       JetBlue Airways Corporation
74443      iShares International Developed Property ETF
74467                            The Ensign Group, Inc.
                               ...                     
2961079                   Theseus Pharmaceuticals, Inc.
2961084                   AXIS Capital Holdings Limited
2961085                                      CBIZ, Inc.
2961086                            FTI Consulting, Inc.
2961087                Gladstone Commercial Corporation
Name: company_name, Length: 1017673, dtype: object

### Duplikate Entfernen

In [8]:
%%time
samples_before = ddf.shape[0]
ddf = ddf.drop_duplicates()
samples_after = ddf.shape[0]
print(f"{samples_before=}, {samples_after=}")

samples_before=1017673, samples_after=1014791
CPU times: user 19.2 s, sys: 386 ms, total: 19.6 s
Wall time: 19.7 s


### Firmennamen-Nachrichtenkörper-Verifikation

In [9]:
def verify_company_name_in_body(df):
    return df.apply(lambda x: bool(re.search(x["short_name"],
                                             x.title + x["body"].replace("( )*\n( )*", " "),
                                             re.IGNORECASE)),
                    axis=1)

In [10]:
%%time
mask = parallelize_dataframe(ddf, verify_company_name_in_body, n_cores=os.cpu_count())

print(f"Around {len(ddf.stocks.unique())} stocks before filtering and {len(ddf[mask].stocks.unique())} after")

/gxfs_work/cau/sunms534/.conda/envs/my_pytorch_env/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 320/320 [00:43<00:00,  7.31it/s]


Around 11383 stocks before filtering and 9899 after
CPU times: user 7.46 s, sys: 11.3 s, total: 18.7 s
Wall time: 51.1 s


In [11]:
# Filter out faulty news
ddf = ddf[mask]

Bis zu 5k Nachrichten pro Firma, z.B. AT&T, was in 13 Jahren ca. einer Nachricht pro Tag entspricht. Wir wollen nicht das eine Firma mit vielen Junk-Nachrichten das Modell dominiert. Wobei das Modell hoffentlich dann auch die Junk-Nachrichten als solche erkennt und ignoriert. Eher wichtig noch einen `staleness`-Faktor, also wie ähnlich die Nachricht zu Vorhergegangenen ist (i.e. Nachrichten desselben Tages oder derselben Woche).

Kategorisieren von Nachrichten (mit Text2Topic, wie Salbrechter?) und eliminieren von Business/Strategic etc.
Im Falle von Text2Topic, versuche Estimates des Unternehmens von Dritten zu unterscheiden.

Wichtig!!! Unterscheide zwischen LERN-Phase und PRODUKTIONS-Phase.
Wir können z.B. CLS-Token in der Produktions-Phase vergleichen, in der Lern-Phase aber noch nicht.

Text2Vec -> Business category evtl. entfernen-> Intrastock variance average

## Make reduced ticker name mapping 

In [12]:
ticker_name_mapper = pd.read_parquet("data_shared/ticker_name_mapper.parquet")

ticker_name_mapper_reduced = ddf[["stocks", "company_name", "short_name"]].drop_duplicates(keep="first")

ticker_name_mapper_reduced.to_parquet("data_shared/ticker_name_mapper_reduced.parquet")

print(f"From {ticker_name_mapper.shape[0]} to {ticker_name_mapper_reduced.shape[0]} tickers (reduced)")

From 11383 to 9899 tickers (reduced)


## Parsing News Bodies


In [13]:
from src.utils.time import convert_timezone
ddf["time"] = ddf["time"].progress_map(lambda x: convert_timezone(pd.to_datetime(x)))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 889200/889200 [00:08<00:00, 105570.59it/s]


In [14]:
sample = ddf.iloc[-1000:, :]

In [15]:
logging.getLogger().setLevel(logging.WARNING)
def filter_df(df):
    return df.apply(filter_body, axis=1) 

In [16]:
%%time
processed_news: pd.Series = parallelize_dataframe(sample, filter_df, n_cores=os.cpu_count())

/gxfs_work/cau/sunms534/.conda/envs/my_pytorch_env/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 320/320 [01:26<00:00,  3.68it/s]


CPU times: user 255 ms, sys: 3.18 s, total: 3.43 s
Wall time: 1min 32s


In [17]:
ddf["parsed_body"] = processed_news

In [31]:
ddf.to_parquet('data/cleaned_sample.parquet') # config.data.benzinga.cleaned

In [48]:
# Analysis
%load_ext autoreload
%autoreload 2
from src.preprocessing.news_parser import remove_patterns

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
ddf.columns

Index(['time', 'stocks', 'author', 'title', 'channels', 'body',
       'inferred_author', 'company_name', 'short_name', 'parsed_body'],
      dtype='object')

In [66]:
ddf.company_name.tail(10)

__null_dask_index__
2960985          Northern Trust Corporation
2960990            Hyatt Hotels Corporation
2960991        Tenet Healthcare Corporation
2961017                   Dana Incorporated
2961056      Martin Midstream Partners L.P.
2961070      Martin Midstream Partners L.P.
2961084       AXIS Capital Holdings Limited
2961085                          CBIZ, Inc.
2961086                FTI Consulting, Inc.
2961087    Gladstone Commercial Corporation
Name: company_name, dtype: object

In [62]:
ddf[["body", "parsed_body"]].tail(10)

,body,parsed_body
__null_dask_index__,,
2960985,"CHICAGO--(BUSINESS WIRE)-- Northern Trust Corporation announced today that it will webcast its third quarter 2012 earnings conference call live on Wednesday, October 17, 2012. The webcast will be accessible on Northern Trust's web site: www.northerntrust.com/financialreleases. The call will be conducted at 11 a.m. CT, following the release that morning of Northern Trust's third quarter 2012 earnings press release. A replay of the webcast will be available for approximately four weeks after the date of the call. Participants will need Windows Media or Adobe Flash software, which may be downloaded free at Northern Trust's web site. Northern Trust's third quarter 2012 earnings press release and presentation materials referred to on the conference call also will be made available at the above web address. **About Northern Trust** Northern Trust Corporation (Nasdaq: NTRS) is a leading provider of investment management, asset and fund administration, banking solutions and fiduciary services for corporations, institutions and affluent individuals worldwide. Northern Trust, a financial holding company based in Chicago, has offices in 18 U.S. states and 16 international locations in North America, Europe, the Middle East and the Asia-Pacific region. As of June 30, 2012, Northern Trust had assets under custody of US$4.6 trillion, and assets under investment management of US$704.3 billion. For more than 120 years, Northern Trust has earned distinction as an industry leader in combining exceptional service and expertise with innovative products and technology. For more information, visit www.northerntrust.com or follow us on Twitter @NorthernTrust. Northern Trust Corporation _Investor Contact:_ Bev Fleming, (312) 444-7811 Beverly_Fleming@ntrs.com or _Media Contact:_ Doug Holt, (312) 557-1571 Doug_Holt@ntrs.com http://www.northerntrust.com","CHICAGO (BUSINESS WIRE) the company announced today that it will webcast its third quarter 2012 earnings conference call live.The call will be conducted at 11 a.m.CT, following the release that morning of the company 's third quarter 2012 earnings press release.A replay of the webcast will be available for approximately four weeks after the date of the call.Participants will need Windows Media or Adobe Flash software, which may be downloaded free at the company 's web site.the company 's third quarter 2012 earnings press release and presentation materials referred to on the conference call also will be made available at the above web address.About the company the company is a leading provider of investment management, asset and fund administration, banking solutions and fiduciary services for corporations, institutions and affluent individuals worldwide.the company , a financial holding company based in Chicago, has offices in 18 U.S.states and 16 international locations in North America, Europe, the Middle East and the Asia Pacific region.As , the company had assets under custody of US$4.6 trillion, and assets under investment management of US$704.3 billion.For more than 120 years, the company has earned distinction as an industry leader in combining exceptional service and expertise with innovative products and technology"
2960990,"CHICAGO--(BUSINESS WIRE)-- Hyatt Hotels Corporation (NYSE: H) announced today that a Hyatt affiliate has entered into a management agreement with Dilony S.A. for a new Hyatt hotel in Montevideo, Uruguay. Hyatt Montevideo will mark the first Hyatt-branded hotel in Uruguay, bringing the total number of announced Hyatt-branded hotels under development in Latin America to 14. Hyatt Montevideo will be the first Hyatt-branded hotel in Uruguay. (C) WSW Montevideo “We are excited to work with Dilony S.A. on the first Hyatt-branded hotel in Uruguay,” said Pat McCudden, Senior Vice President, Real Estate and Development, Hyatt Hotels & Resorts. “This project is an important step in our broader plans to establish Hyatt properties in al

## Filtern von Newstiteln

## Voranstellen von gefilterten Newstiteln an Nachrichtenkörper

# Analyse

## Durschnittlichen Tokenlänge

### Timedeltas zwischen Nachrichtenmeldungen

In [ ]:
import plotly.express as px


In [ ]:
ddf = pd.read_parquet(config.data.benzinga.cleaned)

In [ ]:
ddf = ddf.sort_values("time")
tmp = ddf[["time", "stocks"]]
#### Adding timedeltas to the data frame
news_timedeltas = tmp.groupby("stocks").transform(lambda x: x.diff())
ddf.loc[:, "timedelta"] = news_timedeltas.time.fillna(pd.Timedelta(days=100))

In [ ]:
news_timedeltas = ddf.timedelta
news_timedeltas.iloc[0].components
timedelta_in_minutes = news_timedeltas.apply(lambda x: x.total_seconds() / 60)
px.histogram(timedelta_in_minutes)